In this assignment, you'll work on transparency of Bernoulli naive Bayes. You need a "binary classification" dataset (two classes), and you need all features to be either already binary or you need to binarize them yourself. Do NOT use the "binarize" field in the constructor of BernoulliNB. Each continuous feature should be binarized based on whether it is larger or smaller than its mean. Categorical features that have more than two possible values should use one-hot encoding.  The new features should have proper names according to the binarization. Ask me if you are uncertain about any of this.

1. Choose a binary classification dataset. (The same rules in Assignment 3 applies, except there is one additional rule this time: it has to be a classification dataset).

2. Create a Jupyter notebook. Name it assignment8. It should have several sections. The first section should describe the dataset. The second section should load the dataset. The third section should binarize all the features.

3. If the dataset already comes in train-test split, great. If not, create a train-test split, using 2/3 for train and 1/3 for test.

4. Train a BernouilliNB classifier on the train split; use the default parameters.

5. For the following types of objects, create a section, and print a) the total positive log-evidence,  b) the total negative log-evidence, c) probability distribution, d) top 3 features values that contribute most to the positive evidence, e) top 3 feature values that contribute the most to the negative evidence. Print this information on the following object types
    - The most positive object with respect to the probabilities.
    - The most negative object with respect to the probabilities.
    - The object that has the largest positive evidence.
    - The object that has the largest (in magnitude) negative evidence.
    - The most uncertain object (the probabilities are closest to 0.5)

Upload the notebook to your 595 github repository.

Submit the link using blackboard.

### Description of dataset:

The data is IMDB - review dataset. It's available from the website "http://ai.stanford.edu/~amaas/data/sentiment/". 

The review is binary case which could be positive/negative review. It provides 25,000 reviews for training, and 25,000 reviews for testing. The distribution of class in training and testing is balanced, which means it has 12500 positive reviews and 12500 negative reviews in both training and testing.

In this assignment, we will use countvectorizer which means each token feature will be binary. 

In [1]:
import glob

import numpy as np

from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score

### Load the data 

In [2]:
def load_imdb(path):

    print("Loading the imdb reviews data")

    train_neg_files = glob.glob(path+"/train/neg/*.txt")
    train_pos_files = glob.glob(path+"/train/pos/*.txt")

    train_corpus = []
    y_train = []
    
    for tnf in train_neg_files:
        f = open(tnf, 'r', encoding="utf8")
        line = f.read()
        train_corpus.append(line)
        y_train.append(0)
        f.close()


    for tpf in train_pos_files:
        f = open(tpf, 'r', encoding="utf8")
        line = f.read()
        train_corpus.append(line)
        y_train.append(1)
        f.close()
    
    print("Training Data loaded.")
    print()
    
    test_neg_files = glob.glob(path+"/test/neg/*.txt")
    test_pos_files = glob.glob(path+"/test/pos/*.txt")

    test_corpus = []

    y_test = []

    for tnf in test_neg_files:
        f = open(tnf, 'r', encoding="utf8")
        test_corpus.append(f.read())
        y_test.append(0)
        f.close()

    for tpf in test_pos_files:
        f = open(tpf, 'r', encoding="utf8")
        test_corpus.append(f.read())
        y_test.append(1)
        f.close()

    print("Testing Data loaded.")
    print()
    
    y_train = np.array(y_train)
    y_test = np.array(y_test)

    return train_corpus, test_corpus, y_train, y_test

In [3]:
imdb_path = "./aclImdb"

In [4]:
train_corpus, test_corpus, y_train, y_test = load_imdb(imdb_path)

Loading the imdb reviews data
Training Data loaded.

Testing Data loaded.



### Binarize and vectorize the data into matrix

In [5]:
vectorizer = CountVectorizer(min_df=5, max_df=1.0, binary=True)

X_train = vectorizer.fit_transform(train_corpus)

X_test = vectorizer.transform(test_corpus)

In [6]:
X_train.shape

(25000, 27272)

### Train into the model

In [7]:
clf = BernoulliNB()
clf.fit(X_train, y_train)

y_predict = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)

accuracy_score(y_test, y_predict)

0.82904

### Analysis of model

In [8]:
def get_bias(clf):
    return clf.class_log_prior_[1] - clf.class_log_prior_[0]

In [9]:
# It should be zero since the data is balanced as 50% vs 50%

get_bias(clf)

0.0

In [28]:
# In this case, we only need to calculate the evidence (negative and positive) for all the features 

def evidences(clf, X):
    
    X = X.todense()
    
    X_presence = X.copy()
    
    X_absence = 1 - X
        
    # p(x=absence|y) = 1 - p(x=presence|y)    
    # ln[p(x=absence|y)] = ln[1 - e^ln[p=presence|y]]
    absence_log_prob_ = np.log(1 - np.exp(clf.feature_log_prob_))
    
    # log-ratio => ln[p(x=presence|y=true)/p(x=absence|y=false)] = ln[p(x=presence|y=true)] - ln[p(x=absence|y=true)]
    presence_log_ratios = clf.feature_log_prob_[1] - clf.feature_log_prob_[0]
    
    # log-ratio => ln[p(x=absense|y=true)/p(x=presence|y=false)] = ln[p(x=absense|y=true)] - ln[p(x=presence|y=true)]
    absence_log_ratios = absence_log_prob_[1] - absence_log_prob_[0]
    
    presence_neg_log_ratios = presence_log_ratios * (presence_log_ratios<0)
    presence_pos_log_ratios = presence_log_ratios * (presence_log_ratios>0)
    
    absence_neg_log_ratios = absence_log_ratios * (absence_log_ratios<0)
    absence_pos_log_ratios = absence_log_ratios * (absence_log_ratios>0)
    
    p_neg_evi = np.dot(X_presence, presence_neg_log_ratios)
    p_pos_evi = np.dot(X_presence, presence_pos_log_ratios)
    
    a_neg_evi = np.dot(X_absence, absence_neg_log_ratios)
    a_pos_evi = np.dot(X_absence, absence_pos_log_ratios)
    
    return p_neg_evi.item(0), p_pos_evi.item(0), a_neg_evi.item(0), a_pos_evi.item(0)

In [29]:
def pos_neg_evidence(clf, x_vector):
    
    p_neg_evi, p_pos_evi, a_neg_evi, a_pos_evi = evidences(clf, x_vector)
    
    return p_neg_evi + a_neg_evi, p_pos_evi + a_pos_evi

In [30]:
evidence = []

for i in X_test:
    evidence.append(pos_neg_evidence(clf, i))

In [31]:
evidence = np.asarray(evidence)

evidence.shape

(25000, 2)

In [53]:
evidence[0]

array([-36.61238513,  30.5457996 ])

In [32]:
neg_evidence = evidence[:, 0]
pos_evidence = evidence[:, 1]

#### 1. The most positive object with respect to the probabilities.

In [47]:
idx = np.argmax(y_proba[:,1])

print(y_test[idx])

idx 

0


870

#### 2. The most negative object with respect to the probabilities.

In [48]:
idx = np.argmin(y_proba[:,0])

print(y_test[idx])

idx 

1


20659

#### 3. The object that has the largest positive evidence.

In [54]:
idx = np.argmax(pos_evidence)

print(evidence[idx])

idx 

[-163.3492882   194.84244513]


18112

#### 4. The object that has the largest (in magnitude) negative evidence.

In [55]:
idx = np.argmin(neg_evidence)

print(evidence[idx])

idx 

[-163.3492882   194.84244513]


18112

#### 5. The most uncertain object (the probabilities are closest to 0.5)